In [ ]:
import numpy as np
import tkinter
from PIL import Image
from PIL import ImageTk
from PIL import ImageDraw
from PIL import ImageFont

import tkinter.messagebox
import pickle
import os.path

pickle.dump({'red':1,'yellow':10,'brown':1}, open( "save.p", "wb"))

In [ ]:
def make_square(im, min_size=256, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = min_size #max(min_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [ ]:
class Bookkeeping:
    def __init__(self,master):
        master.master = master
        self.bookkeeping = pickle.load( open( "save.p", "rb" ) )
    def get_names(self):
        return list(self.bookkeeping.keys())
    def get_credit(self,name):
        if name in self.bookkeeping.keys():
            return self.bookkeeping[name]
        else:
            raise IndexError(name + ' not found as name in bookkeeping')
    def add_credit(self,name,amount):
        self.__change_credit(name,np.abs(amount))
    def lower_credit(self,name,amount):
        self.__change_credit(name,-np.abs(amount))
        
    def __change_credit(self,name,amount):
        self.bookkeeping[name] = self.bookkeeping[name]+amount
        pickle.dump(self.bookkeeping, open( "save.p", "wb" ) )
        
    def save_new_person(self):
        if (self.e1.get()==''):
            tkinter.messagebox.showinfo('Input was leeg', 'Vul een naam in, de input was leeg')
        elif (self.e1.get() in self.bookkeeping.keys()):
            tkinter.messagebox.showinfo('Persoon bestaan al', self.e1.get()+" bestaat al en kan niet dubbel worden toegevoegd!")
        else:
            self.bookkeeping[self.e1.get()] = 0
            pickle.dump(self.bookkeeping, open( "save.p", "wb"))
            show_people(self.master,self)
            self.popup.destroy()
        
    def add_person(self):
        self.popup = tkinter.Toplevel()
        self.popup.grab_set()
        self.popup.geometry('400x700')
        tkinter.Label(self.popup, text="Naam").grid(row=0)
#        tkinter.Label(self.popup, text="Last Name").grid(row=1)
        self.e1 = tkinter.Entry(self.popup)
#        self.e2 = tkinter.Entry(self.popup)

        self.e1.grid(row=0, column=1)
#        self.e2.grid(row=1, column=1)

        tkinter.Button(self.popup,text='Save', command=self.save_new_person).grid(row=3, 
                                                       column=1)
    
        tkinter.Button(self.popup,text='Cancel', command=self.quit).grid(row=3, 
                                                       column=2)

    def quit(self):
        self.master.destroy()

In [ ]:

class Person:
    def __init__(self, master,name,bookkeeping,row,col):
        self.master = master
        self.name = name
        self.bookkeeping = bookkeeping
        self.set_person_image()
        
        self.label = tkinter.Label(master, text= name.capitalize()+" heeft nog %i credit(s)"%self.bookkeeping.get_credit(name))
        
        self.button = tkinter.Button(master,image=self.photo, command = self.select_person)
        self.button.grid(row=row,column=col)
        self.label.grid(row=row+1,column=col)
    
    def set_person_image(self):

        if os.path.isfile(self.name+".jpeg"):
            self.image = Image.open(self.name+".jpeg")
            self.image = make_square(self.image)
            self.photo = ImageTk.PhotoImage(self.image)
        else:
            text = Image.new('RGB', (250,250), (255,255,255))
            font = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 25)
            d = ImageDraw.Draw(text)
            d.text((10,10), self.name, font=font, fill=(0,0,0))
            text.save(self.name+".jpeg")
            
            self.image = Image.open(self.name+".jpeg")
            self.image = make_square(self.image)
            self.photo = ImageTk.PhotoImage(self.image)
            
    def update_credit_label(self):
        self.label.config(text = self.name.capitalize()+" heeft nog %i credit(s)"%self.bookkeeping.get_credit(self.name))
        #this.label.pack()
        
    def select_person(self):
        popup = tkinter.Toplevel()
        popup.grab_set()
        popup.geometry('400x700')
        
        credit = self.bookkeeping.get_credit(self.name)
        my_sale = Sales(popup,self,self.bookkeeping)

    def get_name(self):
        return self.name
    
    def quit(self):
        self.master.destroy()

In [ ]:
class Sales:
    def __init__(self,master,person,bookkeeping):
        self.master = master
        self.person = person
        self.bookkeeping = bookkeeping
        
        self.credit = self.bookkeeping.get_credit(person.get_name())
        
        producten = ['bier','cola','fanta']
        amounts = ['5','10']
        
        
        master.title("Verkoop")
        
        self.image = Image.open(self.person.get_name()+".jpeg")
        self.image = self.image.resize((200,200), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(self.image)
        self.label = tkinter.Button(self.master,image=self.photo, command = self.quit).grid(row=0, column=0)
        
        for row,amount in enumerate(amounts):
            Buy_credit(self.master,person,amount,bookkeeping,0,row+1)
        
        if self.credit > 0 :
            for row, product in enumerate(producten):
                Product(self.master,self.person,product,bookkeeping,1,row)
        else:
            tkinter.messagebox.showinfo('Verkoop mislukt', self.person.get_name().capitalize()+" heeft onvoldoende credit.")
            #self.master.destroy()

    def quit(self):
        self.master.destroy()

In [ ]:
class Buy_credit:
    def __init__(self,master,person,amount,bookkeeping,col,row):
        self.master = master
        self.person = person
        self.amount = amount
        self.col = col
        self.row = row
        self.bookkeeping = bookkeeping
        
        self.image = Image.open(amount+".jpeg")
        self.image = self.image.resize((200,200), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(self.image)
        
        self.label = tkinter.Button(master,image=self.photo, command = self.buy).grid(row=row, column=col)
        
    def buy(self):
        #print(self.amount)
        self.bookkeeping.add_credit(self.person.get_name(),int(self.amount))
        #print(self.person)
        self.person.update_credit_label()
        tkinter.messagebox.showinfo('Credit kopen gelukt', self.person.get_name().capitalize()+ " kocht "+self.amount+" credit")
        self.quit()
        
    def quit(self):
        self.master.destroy()
            

In [ ]:
class Product:
    def __init__(self,master,person,product,bookkeeping,col,row):
        self.master = master
        self.person = person
        self.product = product
        self.bookkeeping = bookkeeping
        self.col = col
        self.row = row
        
        self.label = tkinter.Button(master,image=self.photo, command = self.sell).grid(row=row, column=col)
    def picture(self):
        

        if os.path.isfile('filename.txt'):
            self.image = Image.open(product+".jpeg")
            self.image = self.image.resize((200,200), Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(self.image)
        else:
            print ("File not exist")

    def sell(self):
        self.bookkeeping.lower_credit(self.person.get_name(),1)
        self.person.update_credit_label()
        tkinter.messagebox.showinfo('Verkoop gelukt', self.person.get_name().capitalize()+" kocht "+self.product.capitalize())
        print(self.product.capitalize() +' verkocht aan '+self.person.get_name().capitalize() )
        self.quit()
    def quit(self):
        self.master.destroy()

In [ ]:
class show_people:
    def __init__(self,root,bookkeeping):
        self.root = root
        self.bookkeeping = bookkeeping
        number_of_people = len(books.get_names())
        max_num_icons_hor=np.floor(screen_width/250)
        max_num_icons_ver=np.floor(screen_height/250)
        
        #print(number_of_people)
        for ind,person in enumerate(books.get_names()):
            row = int(np.floor((ind+1)/max_num_icons_hor))*2
            col = int((ind+1)%max_num_icons_hor)
            my_gui[person] = Person(root,person,books,row,col)
        

In [ ]:

root = tkinter.Tk()

root.title("Selectie scherm personen")
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

books = Bookkeeping(root)
#root.geometry('500x900')
new_img = Image.open("add_person.png")
new_img = new_img.resize((250,250), Image.ANTIALIAS)
new_photo = ImageTk.PhotoImage(new_img)

my_gui = {}
tkinter.Button(root,image=new_photo,command = books.add_person).grid(row=0,column=0)

show_people(root,books)

root.mainloop()
